# PawsPlace

Im Rahmen unseres Studienmoduls "Geomarketing mit Geodaten" entwickeln wir eine spezialisierte Anwendung, die darauf ausgerichtet ist, Hundehaltern bei der Wohnungssuche zu assistieren, indem sie Grünflächen in der Nähe einer eingegebenen Adresse identifiziert. Diese Anwendung verwendet Geodaten, um die Wohngebiete auf ihre Tauglichkeit für Hundebesitzer zu prüfen, speziell in Bezug auf die Verfügbarkeit und Erreichbarkeit von Grünflächen.


Als erstes werden die benötigten Imports vorgenommen. Wir werden die allgemeinen Reports im oberen Importfenster ansiedeln. Spezifische Imports für Anwendnungen, werden direkt im jeweiligen Kapitel angesiedelt

# Installationen

In [63]:
#!pip install osmnx
#!pip install geopandas
#!pip install folium
#!pip install pandas
!pip install openrouteservice


  Obtaining dependency information for openrouteservice from https://files.pythonhosted.org/packages/67/1a/e6944e4cfd7c5386b15e4b4056084b3c8c676aca0a215d8b507a6cbc1263/openrouteservice-2.3.3-py3-none-any.whl.metadata



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Users\Simone\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
C:\Users\Simone\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedWriter name=3>
  return process_handler(cmd, _system_body)
C:\Users\Simone\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=4>
  return process_handler(cmd, _system_body)
C:\Users\Simone\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io

Nun sind die ersten Installationen erledigt. Als nächstes werden die verwendeten Libraries importiert und OSMNX konfiguriert

In [55]:
import osmnx as ox
import geopandas as gpd
from IPython.display import display, clear_output
 
# Konfigurieren von osmnx
ox.config(log_console=True, use_cache=True)

C:\Users\Simone\AppData\Local\Temp\ipykernel_6724\1196153053.py:6: FutureWarning: The `utils.config` function is deprecated and will be removed in the v2.0.0 release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  ox.config(log_console=True, use_cache=True)


## Eingabe User

Bitte geben Sie Ihre Adresse ein:

Format:

Brandstrasse 1, Schlieren, Switzerland

In [56]:
adresse = input("Bitte geben Sie die gewünschte Adresse ein:")


Wie lange sollen ihre Spaziergänge mit Ihrem Viebeiner maximal dauern

In [57]:
dauerSpaziergang = int(input("Wie lange sollen ihre Spaziergänge mit Ihrem Viebeiner maximal dauern"))


In [58]:
print("Ihre Adresse: ", adresse)
print("Ihre Spaziergangsdauer: ", dauerSpaziergang)

Ihre Adresse:  Landstrasse 11, Turgi, Switzerland
Ihre Spaziergangsdauer:  10


# Funktion zum Laden der Daten und Anzeigen auf einer Karte

In [62]:
# Umrechnen der Spaziergangsdauer in Minuten in Distanz in Metern
# Annahme: Durchschnittliche Gehgeschwindigkeit = 100 Meter pro Minute
distance_in_meters = (int(dauerSpaziergang) * 100)/2

In [70]:
import osmnx as ox
import folium

def load_and_display(adresse, dauerSpaziergang):
    try:
        # Geocode Adresse
        location = ox.geocoder.geocode(adresse)
                
        # Suche nach Grünflächen in der Nähe
        tags = {'landuse': ['greenfield', 'grass','forest'],
        'leisure': 'park'}

        # Suche nach Grünflächen gemäss Userinput
        gdf = ox.features_from_point(location, tags, dist=distance_in_meters)
        
        # Karte erstellen
        m = folium.Map(location=[location[0], location[1]], zoom_start=14)
        folium.Marker([location[0], location[1]], popup='Eingegebene Adresse').add_to(m)
        
        # Grünflächen hinzufügen/markieren
        for _, row in gdf.iterrows():
            if row['geometry'].geom_type == 'Polygon':
                folium.Polygon(
                    locations=[(lat, lon) for lon, lat in row['geometry'].exterior.coords],
                    color='green', fill=True, fill_color='green'
                ).add_to(m)
        
        display(m)
    except Exception as e:
        print(f"Ein Fehler ist aufgetreten: {e}")

# Aufruf der Funktion mit Benutzereingaben
load_and_display(adresse, dauerSpaziergang)


Nun wird die Route berechnet

In [74]:
def read_api_key(file_path):
    with open(file_path, 'r') as file:
        api_key = file.readline().strip()
    return api_key

file_path = 'key.txt'
api_key = read_api_key(file_path)
print("Der gelesene API Key ist:", api_key)


Der gelesene API Key ist: 5b3ce3597851110001cf6248e9b328d2e57f4482bb6e1a89a697e63e


In [75]:
import openrouteservice
from openrouteservice import convert
import geopandas as gpd
 
 ##    wegggggg
def fetch_green_spaces(area):
    """Lade Grünflächen für ein bestimmtes Gebiet über OSMnx."""
    # Definieren Sie den Typ der gewünschten Grünflächen
    tags = {'leisure': 'park', 'landuse': ['forest', 'meadow']}
    green_spaces = ox.geometries_from_place(area, tags)
    return green_spaces
 

def calculate_isochrones(start_coords, minutes, travel_mode='driving-car', api_key= api_key):
    client = openrouteservice.Client(key=api_key)
    params = {
        'locations': [start_coords],  # Stellen Sie sicher, dass dies eine Liste von Listen ist
        'profile': travel_mode,
        'range': [minutes * 60],  # Zeit in Sekunden
        'range_type': 'time',
        'intersections': 'true'
    }
    try:
        isochrones = client.isochrones(params)
        return isochrones
    except Exception as e:
        print(f"An error occurred: {e}")
 
# Beispiel-Koordinaten und Zeit in Minuten
coords = [8.681495, 49.41461]  # Koordinaten von Heidelberg, Deutschland
minutes = 10  # 10 Minuten Radius
 
# Versuchen Sie, die Isochronen zu generieren
isochrones = calculate_isochrones(coords, minutes, api_key=api_key)
print(isochrones)
 
def main():
    start_coords = [8.681495, 49.41461]
    minutes = dauerSpaziergang
    api_key = '5b3ce3597851110001cf6248e9b328d2e57f4482bb6e1a89a697e63e'

    # Grünflächen laden
    area = "Heidelberg, Germany"
    green_spaces = fetch_green_spaces(area)
    # Isochronen berechnen
    isochrones = calculate_isochrones(start_coords, minutes, api_key=api_key)
    isochrones_gdf = gpd.GeoDataFrame.from_features(isochrones['features'])
    # Überschneidung zwischen Isochronen und Grünflächen prüfen
    intersection = gpd.overlay(isochrones_gdf, green_spaces, how='intersection')
    # Karte erzeugen
    map = folium.Map(location=start_coords, zoom_start=12)
    folium.GeoJson(intersection, style_function=lambda x: {'fillColor': 'green'}).add_to(map)
    folium.GeoJson(isochrones_gdf, style_function=lambda x: {'fillColor': 'blue', 'fillOpacity': 0.5}).add_to(map)
    return map
 
# Karte generieren und anzeigen
map_result = main()
map_result.save('isochrone_green_spaces_map.html')

An error occurred: 400 ({'error': {'code': 3002, 'message': "Parameter 'locations' has incorrect value or format."}, 'info': {'engine': {'build_date': '2024-03-21T13:55:54Z', 'version': '8.0.0'}, 'timestamp': 1715351344531}})
None


C:\Users\Simone\AppData\Local\Temp\ipykernel_6724\1153527707.py:10: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  green_spaces = ox.geometries_from_place(area, tags)


An error occurred: 400 ({'error': {'code': 3002, 'message': "Parameter 'locations' has incorrect value or format."}, 'info': {'engine': {'build_date': '2024-03-21T13:55:54Z', 'version': '8.0.0'}, 'timestamp': 1715351346638}})


TypeError: 'NoneType' object is not subscriptable

## Anbindung PostGRESQL

In [1]:
import psycopg2


# Verbindungsdaten
host = "localhost"
dbname = "deine_datenbank"
user = "dein_user"
password = "12345678"

# Verbindung aufbauen
conn = psycopg2.connect(
    host=host,
    dbname=dbname,
    user=user,
    password=password
)

# Einen Cursor erstellen
cur = conn.cursor()

# Eine SQL-Abfrage ausführen
cur.execute("SELECT * FROM deine_tabelle LIMIT 5;")

# Daten abrufen
rows = cur.fetchall()

for row in rows:
    print(row)

# Ressourcen freigeben
cur.close()
conn.close()


ModuleNotFoundError: No module named 'psycopg2'

## Eingabe User

Geben Sie die Adresse ein, an der wir nach Grünflächen suchen sollen?

Vielen Dank für Ihre Angabe.

## Verarbeitung Daten

Nun werden die Daten verarbeitet

## Anzeige Daten

Hier wird die Map ausgegeben

Hier werden Berechnungen für Entfernungen ausgegeben

In diesem Bereich können Filter definiert werden für die Spaziergang länge

## Evaluation

Zusammenfassung der Ergebnisse